<a href="https://colab.research.google.com/github/vidushimeel/BreastCancerTrial/blob/spell-branch/Copy_of_breastCancerTrial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing necessary libraries:** 
numpy, pandas, matlotlib, joblib, sklearn

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

from joblib import dump, load

In [ ]:
cols = ['id', 'diagnosis','radius-mean', 'texture-mean', 'perimeter-mean', 'area-mean', 'smoothness-mean', 'compactness-mean', 'concavity-mean', 'concave-points-mean', 'symmetry-mean', 'fractal-dimension-mean', 'radius-se', 'texture-se', 'perimeter-se', 'area-se', 'smoothness-se', 'compactness-se', 'concavity-se', 'concave-points-se', 'symmetry-se', 'fractal-dimension-se', 'radius-ex', 'texture-ex', 'perimeter-ex', 'area-ex', 'smoothness-ex', 'compactness-ex', 'concavity-ex', 'concave-points-ex', 'symmetry-ex', 'fractal-dimension-ex']
df = pd.read_csv('https://ftp.cs.wisc.edu/math-prog/cpo-dataset/machine-learn/cancer/WDBC/WDBC.dat', names=cols)
df.head()

,id,diagnosis,radius-mean,texture-mean,perimeter-mean,area-mean,smoothness-mean,compactness-mean,concavity-mean,concave-points-mean,symmetry-mean,fractal-dimension-mean,radius-se,texture-se,perimeter-se,area-se,smoothness-se,compactness-se,concavity-se,concave-points-se,symmetry-se,fractal-dimension-se,radius-ex,texture-ex,perimeter-ex,area-ex,smoothness-ex,compactness-ex,concavity-ex,concave-points-ex,symmetry-ex,fractal-dimension-ex
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
#importing our cancer dataset
x = df.iloc[:, 2:32]
y = df.iloc[:, 1]

In [ ]:
x.head() #what we are training model on

,radius-mean,texture-mean,perimeter-mean,area-mean,smoothness-mean,compactness-mean,concavity-mean,concave-points-mean,symmetry-mean,fractal-dimension-mean,radius-se,texture-se,perimeter-se,area-se,smoothness-se,compactness-se,concavity-se,concave-points-se,symmetry-se,fractal-dimension-se,radius-ex,texture-ex,perimeter-ex,area-ex,smoothness-ex,compactness-ex,concavity-ex,concave-points-ex,symmetry-ex,fractal-dimension-ex
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
y.head() #what we are trying to predict (vars)

0    M
1    M
2    M
3    M
4    M
Name: diagnosis, dtype: object

In [ ]:
#find missing or null data points 
df.isnull().sum()
df.isna().sum()

id                        0
diagnosis                 0
radius-mean               0
texture-mean              0
perimeter-mean            0
area-mean                 0
smoothness-mean           0
compactness-mean          0
concavity-mean            0
concave-points-mean       0
symmetry-mean             0
fractal-dimension-mean    0
radius-se                 0
texture-se                0
perimeter-se              0
area-se                   0
smoothness-se             0
compactness-se            0
concavity-se              0
concave-points-se         0
symmetry-se               0
fractal-dimension-se      0
radius-ex                 0
texture-ex                0
perimeter-ex              0
area-ex                   0
smoothness-ex             0
compactness-ex            0
concavity-ex              0
concave-points-ex         0
symmetry-ex               0
fractal-dimension-ex      0
dtype: int64

In [ ]:
#replaces missing values in dataset 
from sklearn.impute import SimpleImputer #from the sklearn.impute, import SimpleImputer class  
imputer = SimpleImputer(missing_values =  np.nan, strategy = 'mean') #var imputer = object of SimpleImputer class 
#SimpleImputer ()
imputer.fit(x) #will look for all the missing values in the age and salary column, exclude String columns 
x = imputer.transform(x)#will replace

In [ ]:
#encode the Y, which is categorical 
#convert M/B to 1 and 0 
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
y = le.fit_transform(y) #dont need to cast as num py array bc dependent variabel does not have to be numpy array 
print (y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0
 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 1 0 0 1 

In [ ]:
#split into training and testing 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [ ]:
#feature scaling 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() #the formula 
#only apply feature scaling to numerical values, leave dummy variables alone 
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
#Using RandomForestClassifier method of ensemble class to use Random Forest Classification algorithm

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm)

[[89  1]
 [ 1 52]]


In [ ]:
#89 true pos. 
#1 false pos. 
#1 false neg.
#52 true neg. 
#total tests = 143
#to get accuracy rate, do 141/143 = 98.6%